In [1]:
import pandas as pd

In [2]:
df = pd.read_excel(r"C:\Users\Eqra\Desktop\Quran_Project\processing.xlsx")
df

,الكلمة,الكلمة بدون تشكيل
0,بِسْمِ,بسم
1,اللَّهِ,الله
2,الرَّحْمَٰنِ,الرحمن
3,الرَّحِيمِ,الرحيم
4,الْحَمْدُ,الحمد
...,...,...
78243,صُدُورِ,صدور
78244,النَّاسِ,الناس
78245,مِنَ,من
78246,الْجِنَّةِ,الجنة


In [3]:
df['بدون_تشكيل'] = df['الكلمة بدون تشكيل']

df = df.drop(columns=['الكلمة بدون تشكيل'])

cols = df.columns.tolist()
cols.insert(1, cols.pop(cols.index('بدون_تشكيل')))

df = df[cols]
df

,الكلمة,بدون_تشكيل
0,بِسْمِ,بسم
1,اللَّهِ,الله
2,الرَّحْمَٰنِ,الرحمن
3,الرَّحِيمِ,الرحيم
4,الْحَمْدُ,الحمد
...,...,...
78243,صُدُورِ,صدور
78244,النَّاسِ,الناس
78245,مِنَ,من
78246,الْجِنَّةِ,الجنة


In [4]:
tanween = "ًٌٍ"

# المرور على الكلمات
for i in range(len(df)):
    word = str(df.loc[i, "الكلمة"]).strip() if pd.notnull(df.loc[i, "الكلمة"]) else ""

    if len(word) > 0 and (word[-1] in tanween or (len(word) > 1 and word[-2] in tanween)):
        # حفظ الكلمة المحذوفة
        df.loc[i, "المحذوف"] = word
        # تصنيفها كاسم
        df.loc[i, "tag"] = "اسم"
        # حذفها من العمود الأصلي
        df.loc[i, "الكلمة"] = None

print(df.head())


         الكلمة بدون_تشكيل المحذوف  tag
0        بِسْمِ        بسم     NaN  NaN
1       اللَّهِ       الله     NaN  NaN
2  الرَّحْمَٰنِ     الرحمن     NaN  NaN
3    الرَّحِيمِ     الرحيم     NaN  NaN
4     الْحَمْدُ      الحمد     NaN  NaN


In [5]:
df['tag'].value_counts()

tag
اسم    8894
Name: count, dtype: int64

In [6]:
import pandas as pd

# قراءة الملف الخارجي
external_df = pd.read_excel(r"C:\Users\Eqra\Desktop\Quran_Project\Data\Data\arabic_jamida_names.xlsx")

# أخذ العمود الثالث (index 2) وتحويله لقائمة بعد إزالة أي فراغات
external_words = external_df.iloc[:, 2].dropna().astype(str).str.strip().tolist()

# إنشاء عمود 'tag' في df إذا مش موجود
if 'tag' not in df.columns:
    df['tag'] = None

# وضع 'اسم' للكلمات الموجودة في external_words
df.loc[df['بدون_تشكيل'].isin(external_words), 'tag'] = 'اسم'

# حذف الكلمات المصنفة من العمود الأصلي 'بدون_تشكيل' (يمكنك تعديل العمود إذا تريد)
df.loc[df['tag'] == 'اسم', 'بدون_تشكيل'] = None

# إذا حابب تخزن الكلمات المحذوفة في عمود جديد
df['المحذوف'] = df.apply(lambda x: x['بدون_تشكيل'] if x['tag']=='اسم' else None, axis=1)

print(df.head())

         الكلمة بدون_تشكيل المحذوف  tag
0        بِسْمِ        بسم    None  NaN
1       اللَّهِ       None    None  اسم
2  الرَّحْمَٰنِ       None    None  اسم
3    الرَّحِيمِ       None    None  اسم
4     الْحَمْدُ      الحمد    None  NaN


In [7]:
print(df[df['tag'].notnull()])

             الكلمة بدون_تشكيل المحذوف  tag
1           اللَّهِ       None    None  اسم
2      الرَّحْمَٰنِ       None    None  اسم
3        الرَّحِيمِ       None    None  اسم
8      الرَّحْمَٰنِ       None    None  اسم
9        الرَّحِيمِ       None    None  اسم
...             ...        ...     ...  ...
78221          None       None    None  اسم
78225       اللَّهِ       None    None  اسم
78226  الرَّحْمَٰنِ       None    None  اسم
78227    الرَّحِيمِ       None    None  اسم
78240       الَّذِي       None    None  اسم

[16702 rows x 4 columns]


In [8]:
df['tag'].value_counts()

tag
اسم    16702
Name: count, dtype: int64

In [9]:
import numpy as np

# قائمة أحرف الجر بدون الباء واللام
arabic_prepositions = [
    "من", "إلى", "عن", "على", "في", "الكاف", "حتى",
    "خلا", "عدا", "ماعدا", "مذ", "منذ", "مع", "غير", "ربّ"
]

# التأكد من وجود عمود 'tag'
if 'tag' not in df.columns:
    df['tag'] = None

# المرور على الكلمات
for i in range(len(df) - 1):  # -1 عشان ما يطلع خارج الحدود
    current_word = str(df.loc[i, "بدون_تشكيل"]).strip()

    if current_word in arabic_prepositions:
        # إذا فيه تصنيف سابق في tag، نضيف مع فاصلة
        # حفظ الكلمة المحذوفة في عمود 'المحذوف' وحذفها من عمود 'بدون_تشكيل'
        df.loc[i + 1, "المحذوف"] = df.loc[i + 1, "بدون_تشكيل"]
        df.loc[i + 1, "بدون_تشكيل"] = None
        # تصنيف حرف الجر في عمود tag
        df.loc[i + 1, "tag"] = "حرف جر"

print(df.head())

         الكلمة بدون_تشكيل المحذوف  tag
0        بِسْمِ        بسم    None  NaN
1       اللَّهِ       None    None  اسم
2  الرَّحْمَٰنِ       None    None  اسم
3    الرَّحِيمِ       None    None  اسم
4     الْحَمْدُ      الحمد    None  NaN


In [10]:
df[df['tag'] == 'حرف جر']

,الكلمة,بدون_تشكيل,المحذوف,tag
25,الْمَغْضُوبِ,None,المغضوب,حرف جر
57,قَبْلِكَ,None,قبلك,حرف جر
63,None,None,None,حرف جر
65,رَبِّهِمْ,None,ربهم,حرف جر
83,قُلُوبِهِمْ,None,قلوبهم,حرف جر
...,...,...,...,...
78206,شَرِّ,None,شر,حرف جر
78218,الْعُقَدِ,None,None,حرف جر
78237,شَرِّ,None,شر,حرف جر
78243,صُدُورِ,None,صدور,حرف جر


In [11]:
print(df[df['المحذوف'].notnull()])


             الكلمة بدون_تشكيل  المحذوف     tag
25     الْمَغْضُوبِ       None  المغضوب  حرف جر
57         قَبْلِكَ       None     قبلك  حرف جر
65        رَبِّهِمْ       None     ربهم  حرف جر
83      قُلُوبِهِمْ       None   قلوبهم  حرف جر
95          يَقُولُ       None     يقول  حرف جر
...             ...        ...      ...     ...
78174      جِيدِهَا       None    جيدها  حرف جر
78206         شَرِّ       None       شر  حرف جر
78237         شَرِّ       None       شر  حرف جر
78243       صُدُورِ       None     صدور  حرف جر
78246    الْجِنَّةِ       None    الجنة  حرف جر

[4000 rows x 4 columns]


In [12]:
df['المحذوف'].count()

np.int64(4000)

In [13]:
df['tag'].value_counts()

tag
اسم       15223
حرف جر     5479
Name: count, dtype: int64

In [14]:
df.to_csv(r"C:\Users\Eqra\Desktop\Quran_Project\processing.2.csv", index=False, encoding='utf-8-sig')